**Topic Modelling**

In [1]:
!pip install -U gensim

Requirement already up-to-date: gensim in /opt/anaconda3/lib/python3.8/site-packages (3.8.3)


In [2]:
import os
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import nltk
nltk.download('wordnet')
import gensim
from gensim import corpora
import pandas as pd
import pickle

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andreaprenner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#Set WD
os.chdir("/Users/andreaprenner/Desktop/Master_UvA/1. Semester/FundamentalsOfDS/Assignment1")

In [40]:
data_formatted = pd.read_pickle("twitter_data_classified_23k.pickle")

In [9]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

In [10]:
stopwords = stopwords.words('english')

In [57]:
#Data cleaning - removing punctuation from text; still need to remove hashes
data_formatted['text_topicmodel'] = data_formatted['text'].str.replace('@[^\s]+','')
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].str.replace('[^\w\s]','')
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].str.replace('\[.*?\]','')
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].str.replace('[‘’“”…]','')
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].str.replace('\w*\d\w*','')
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].apply(lambda x: x.lower())
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].str.replace("trump", "")
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].str.replace("clinton", "")
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].apply(lambda x: x.split())
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].apply(lambda x: [item for item in x if item not in stopwords])
data_formatted['text_topicmodel'] = data_formatted['text_topicmodel'].apply(lemmatize_text)
data_formatted['text_topicmodel']

0                   [insult, democracy, tweet, defeat]
0    [morningjoe, alleged, child, rapist, lapdog, a...
0                               [foley, sex, offender]
0    [dont, come, admit, youre, bag, since, day, on...
0              [clip, said, declare, victory, get, id]
                           ...                        
0      [thank, clarifying, hasnt, convicted, anything]
0                                  [thanks, good, job]
0    [scoop, n, gen, flynn, supporter, translater, ...
0    [like, love, poorly, educated, supporter, igno...
0    [half, supporter, basket, deplorables, crooked...
Name: text_topicmodel, Length: 20846, dtype: object

In [58]:
#Filter out the rows where Trump and Clinton both are mentioned
data_formatted = data_formatted[~((data_formatted['Trump'] == True) & (data_formatted['Clinton'] == True))]

In [59]:
#Create data set for trump vs clinton
trump_mentioned_positive = data_formatted[(data_formatted['Trump'] == True) & (data_formatted['classification_polarity'] == 'positive')]
trump_mentioned_negative = data_formatted[(data_formatted['Trump'] == True) & (data_formatted['classification_polarity'] == 'negative')]
clinton_mentioned_positive = data_formatted[(data_formatted['Clinton'] == True) & (data_formatted['classification_polarity'] == 'positive')]
clinton_mentioned_negative = data_formatted[(data_formatted['Clinton'] == True) & (data_formatted['classification_polarity'] == 'negative')]

**Trump mentioned / Positive Polarity Score / All words**

In [44]:
#Filter out rows where text doesnt contain any entries after removing tags and hashes
trump_mentioned_positive = trump_mentioned_positive[trump_mentioned_positive['text_topicmodel'].map(lambda d: len(d)) > 0]
trump_text = trump_mentioned_positive.text_topicmodel.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
# Creating the object for LDA model using gensim library
Lda_trump = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_trump = Lda_trump(doc_term_matrix_trump, num_topics=2, id2word = dictionary_trump, passes=100)
topics_trump = ldamodel_trump.print_topics(num_topics=2, num_words=8)
topics_trump

[(0,
  '0.019*"trump" + 0.018*"amp" + 0.011*"u" + 0.008*"nevertrump" + 0.007*"hillary" + 0.006*"love" + 0.006*"maga" + 0.006*"clinton"'),
 (1,
  '0.034*"trump" + 0.010*"like" + 0.008*"u" + 0.008*"great" + 0.008*"america" + 0.008*"know" + 0.007*"one" + 0.006*"youre"')]

**Trump mentioned / Negative Polarity Score / All words**

In [93]:
trump_mentioned_negative = trump_mentioned_negative[trump_mentioned_negative['text_topicmodel'].map(lambda d: len(d)) > 0]
trump_text = trump_mentioned_negative.text_topicmodel.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
# Creating the object for LDA model using gensim library
Lda_trump = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_trump = Lda_trump(doc_term_matrix_trump, num_topics=2, id2word = dictionary_trump, passes=100)
topics_trump = ldamodel_trump.print_topics(num_topics=2, num_words=8)
topics_trump

[(0,
  '0.033*"trump" + 0.010*"today" + 0.009*"pressure" + 0.009*"rain" + 0.009*"tempcrab" + 0.009*"forecast" + 0.009*"orchard" + 0.007*"fine"'),
 (1,
  '0.020*"trump" + 0.015*"u" + 0.010*"amp" + 0.009*"dont" + 0.008*"get" + 0.008*"like" + 0.008*"people" + 0.008*"hillary"')]

**Clinton mentioned / Positive Polarity Score / All words**

In [95]:
#Filter out rows where text doesnt contain any entries after removing tags and hashes
clinton_mentioned_positive = clinton_mentioned_positive[clinton_mentioned_positive['text_topicmodel'].map(lambda d: len(d)) > 0]
clinton_text = clinton_mentioned_positive.text_topicmodel.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
# Creating the object for LDA model using gensim library
Lda_clinton = gensim.models.ldamodel.LdaModel
# Running and Trainign LDA model on the document term matrix.
ldamodel_clinton = Lda_clinton(doc_term_matrix_clinton, num_topics=2, id2word = dictionary_clinton, passes=100)
topics_clinton = ldamodel_clinton.print_topics(num_topics=10, num_words=4)
topics_clinton

[(0, '0.015*"imwithher" + 0.015*"hillary" + 0.014*"trump" + 0.012*"amp"'),
 (1, '0.011*"trump" + 0.010*"amp" + 0.009*"u" + 0.008*"hillary"')]

**Clinton mentioned / Negative Polarity Score / All words**

In [97]:
#Filter out rows where text doesnt contain any entries after removing tags and hashes
clinton_mentioned_negative = clinton_mentioned_negative[clinton_mentioned_negative['text_topicmodel'].map(lambda d: len(d)) > 0]
clinton_text = clinton_mentioned_positive.text_topicmodel.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
# Creating the object for LDA model using gensim library
Lda_clinton = gensim.models.ldamodel.LdaModel
# Running and Training LDA model on the document term matrix.
ldamodel_clinton = Lda_clinton(doc_term_matrix_clinton, num_topics=2, id2word = dictionary_clinton, passes=100)
topics_clinton = ldamodel_clinton.print_topics(num_topics=10, num_words=4)
topics_clinton

[(0, '0.013*"amp" + 0.012*"hillary" + 0.012*"u" + 0.011*"trump"'),
 (1, '0.015*"trump" + 0.013*"imwithher" + 0.011*"hillary" + 0.011*"clinton"')]

**Just using nouns in Topic Modelling**

In [45]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag
import numpy as np

**Trump mentioned / Positive Polarity Score / Only Nouns**

In [60]:
#Apply function that determines the type of each word
trump_mentioned_text = trump_mentioned_positive['text_topicmodel'].apply(lambda x: nltk.pos_tag(x))
trump_mentioned_text

0    [(insult, NN), (democracy, NN), (tweet, NN), (...
0    [(morningjoe, NNS), (alleged, VBD), (child, JJ...
0    [(dont, JJ), (come, JJ), (admit, NN), (youre, ...
0    [(clip, NN), (said, VBD), (declare, JJ), (vict...
0    [(word, NN), (consequence, NN), (loser, NN), (...
                           ...                        
0    [(remember, VB), (slick, JJ), (willie, NN), (s...
0    [(somebody, NN), (smoking, VBG), (something, N...
0    [(think, VB), (snort, NN), (coke, NNS), (take,...
0               [(thanks, NNS), (good, JJ), (job, NN)]
0    [(like, IN), (love, JJS), (poorly, RB), (educa...
Name: text_topicmodel, Length: 7681, dtype: object

In [61]:
is_noun = lambda pos: pos[:2] == 'NN'

In [62]:
#Filter out the nouns for each row
trump_mentioned_nouns = pd.DataFrame(np.zeros((len(trump_mentioned_text), 1)))
trump_mentioned_nouns.columns = ['Text']
for i in range(0, len(trump_mentioned_text)):
    nouns = [word for (word, pos) in trump_mentioned_text.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    trump_mentioned_nouns.iloc[i] = nouns

In [63]:
#Split the Text 
trump_mentioned_nouns['Text'] = trump_mentioned_nouns['Text'].apply(lambda x: x.split())

In [64]:
trump_text = trump_mentioned_nouns.Text.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
Lda_trump = gensim.models.ldamodel.LdaModel
ldamodel_trump_1 = Lda_trump(doc_term_matrix_trump, num_topics=10, id2word = dictionary_trump, passes=2)
ldamodel_trump_1.print_topics(num_topics=10, num_words=5)
#for idx, topic in ldamodel_trump.print_topics(-1):
 #   print('Topic: {} Word: {}'.format(idx, topic))

[(0,
  '0.039*"job" + 0.029*"president" + 0.027*"word" + 0.025*"putin" + 0.019*"everyone"'),
 (1,
  '0.031*"time" + 0.027*"wall" + 0.024*"idiot" + 0.019*"day" + 0.016*"dump"'),
 (2,
  '0.029*"way" + 0.026*"truth" + 0.020*"man" + 0.019*"candidate" + 0.017*"train"'),
 (3,
  '0.033*"vote" + 0.027*"love" + 0.024*"guy" + 0.022*"thank" + 0.020*"campaign"'),
 (4,
  '0.053*"america" + 0.027*"im" + 0.018*"maga" + 0.013*"time" + 0.013*"amp"'),
 (5,
  '0.042*"donald" + 0.030*"youre" + 0.025*"r" + 0.019*"maga" + 0.019*"obama"'),
 (6,
  '0.028*"country" + 0.022*"poll" + 0.017*"pay" + 0.015*"idea" + 0.014*"shes"'),
 (7,
  '0.053*"tax" + 0.029*"plan" + 0.028*"look" + 0.024*"return" + 0.019*"release"'),
 (8,
  '0.054*"people" + 0.025*"supporter" + 0.024*"racist" + 0.019*"amp" + 0.018*"support"'),
 (9,
  '0.024*"mexico" + 0.022*"lie" + 0.021*"leader" + 0.021*"win" + 0.019*"woman"')]

**Trump mentioned / Negative Polarity Score / Only Nouns**

In [73]:
#Apply function that determines the type of each word
trump_mentioned_text = trump_mentioned_negative['text_topicmodel'].apply(lambda x: nltk.pos_tag(x))
trump_mentioned_text

0    [(msnbc, NN), (cnn, NN), (mention, NN), (fox, ...
0    [(trueanother, RB), (fasle, JJ), (coverage, NN...
0    [(didnt, NN), (sound, NN), (sarcastic, JJ), (i...
0    [(isi, NN), (trouble, NN), (flip, NN), (get, N...
0    [(trying, VBG), (lose, JJ), (cant, JJ), (wait,...
                           ...                        
0    [(manny, JJ), (alotta, NN), (people, NNS), (sa...
0    [(last, JJ), (day, NN), (strong, JJ), (man, NN...
0    [(joining, VBG), (administration, NN), (rumore...
0    [(mean, JJ), (speech, NN), (home, NN), (run, N...
0    [(totem, NN), (pole, JJ), (little, JJ), (bill,...
Name: text_topicmodel, Length: 5636, dtype: object

In [74]:
#Filter out the nouns for each row
trump_mentioned_nouns = pd.DataFrame(np.zeros((len(trump_mentioned_text), 1)))
trump_mentioned_nouns.columns = ['Text']
for i in range(0, len(trump_mentioned_text)):
    nouns = [word for (word, pos) in trump_mentioned_text.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    trump_mentioned_nouns.iloc[i] = nouns

In [75]:
#Split the Text 
trump_mentioned_nouns['Text'] = trump_mentioned_nouns['Text'].apply(lambda x: x.split())

In [76]:
trump_text = trump_mentioned_nouns.Text.tolist()
dictionary_trump = corpora.Dictionary(trump_text)
doc_term_matrix_trump = [dictionary_trump.doc2bow(doc) for doc in trump_text]
ldamodel_trump_2 = Lda_trump(doc_term_matrix_trump, num_topics=10, id2word = dictionary_trump, passes=2)
ldamodel_trump_2.print_topics(num_topics=10, num_words=5)

[(0,
  '0.030*"president" + 0.021*"work" + 0.020*"racist" + 0.017*"need" + 0.015*"debate"'),
 (1,
  '0.027*"speech" + 0.027*"immigration" + 0.018*"dont" + 0.015*"day" + 0.013*"donald"'),
 (2,
  '0.032*"lie" + 0.022*"didnt" + 0.016*"amp" + 0.015*"state" + 0.014*"mexico"'),
 (3,
  '0.045*"maga" + 0.019*"world" + 0.016*"poll" + 0.015*"shit" + 0.014*"show"'),
 (4,
  '0.018*"putin" + 0.018*"thing" + 0.014*"news" + 0.013*"think" + 0.013*"anyone"'),
 (5,
  '0.025*"amp" + 0.020*"wall" + 0.020*"get" + 0.020*"medium" + 0.013*"hillary"'),
 (6,
  '0.025*"hate" + 0.024*"doesnt" + 0.023*"anything" + 0.017*"thats" + 0.015*"people"'),
 (7,
  '0.038*"donald" + 0.029*"vote" + 0.027*"year" + 0.025*"talk" + 0.020*"im"'),
 (8,
  '0.021*"today" + 0.018*"plan" + 0.017*"amp" + 0.014*"candidate" + 0.014*"something"'),
 (9,
  '0.033*"nothing" + 0.033*"people" + 0.029*"tax" + 0.016*"voter" + 0.015*"country"')]

**Clinton mentioned / Positive Polarity Score / Only Nouns**

In [77]:
#Apply function that determines the type of each word
clinton_mentioned_text = clinton_mentioned_positive['text_topicmodel'].apply(lambda x: nltk.pos_tag(x))
clinton_mentioned_text

0             [(foley, NN), (sex, NN), (offender, NN)]
0    [(press, NN), (advocate, NN), (lame, JJ), (duc...
0          [(yes, RB), (exactly, RB), (imwithher, RB)]
0    [(attacking, VBG), (gave, VBD), (pas, JJ), (pa...
0    [(american, JJ), (people, NNS), (right, RB), (...
                           ...                        
0    [(unfit, JJ), (medically, RB), (become, VBN), ...
0    [(totally, RB), (different, JJ), (statute, NN)...
0    [(real, JJ), (hillary, JJ), (every, DT), (time...
0                         [(aug, NN), (imwithher, NN)]
0    [(copy, NN), (speech, NN), (made, VBD), (print...
Name: text_topicmodel, Length: 2889, dtype: object

In [78]:
#Filter out the nouns for each row
clinton_mentioned_nouns = pd.DataFrame(np.zeros((len(clinton_mentioned_text), 1)))
clinton_mentioned_nouns.columns = ['Text']
for i in range(0, len(clinton_mentioned_text)):
    nouns = [word for (word, pos) in clinton_mentioned_text.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    clinton_mentioned_nouns.iloc[i] = nouns

In [79]:
#Split the Text 
clinton_mentioned_nouns['Text'] = clinton_mentioned_nouns['Text'].apply(lambda x: x.split())

In [80]:
clinton_text = clinton_mentioned_nouns.Text.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
Lda_clinton = gensim.models.ldamodel.LdaModel
ldamodel_clinton_1 = Lda_clinton(doc_term_matrix_clinton, num_topics=10, id2word = dictionary_clinton, passes=2)
ldamodel_clinton_1.print_topics(num_topics=10, num_words=5)

[(0,
  '0.028*"people" + 0.020*"email" + 0.018*"vote" + 0.016*"plan" + 0.012*"obama"'),
 (1,
  '0.024*"time" + 0.020*"press" + 0.019*"people" + 0.018*"lie" + 0.017*"medium"'),
 (2,
  '0.026*"hillary" + 0.024*"day" + 0.013*"corrupt" + 0.012*"video" + 0.009*"election"'),
 (3,
  '0.020*"way" + 0.019*"president" + 0.017*"dont" + 0.016*"love" + 0.015*"time"'),
 (4,
  '0.051*"amp" + 0.031*"r" + 0.028*"imwithher" + 0.019*"u" + 0.016*"supporter"'),
 (5,
  '0.021*"jail" + 0.016*"america" + 0.015*"campaign" + 0.011*"cant" + 0.010*"fund"'),
 (6,
  '0.017*"thats" + 0.014*"health" + 0.012*"show" + 0.012*"r" + 0.011*"donald"'),
 (7,
  '0.019*"foundation" + 0.012*"pay" + 0.010*"vet" + 0.009*"time" + 0.009*"amp"'),
 (8,
  '0.071*"imwithher" + 0.015*"liar" + 0.013*"woman" + 0.010*"crookedhillary" + 0.010*"isi"'),
 (9,
  '0.029*"shes" + 0.025*"question" + 0.023*"amp" + 0.015*"hell" + 0.013*"war"')]

**Clinton mentioned / Negative Polarity Score / Only Nouns**

In [81]:
#Apply function that determines the type of each word
clinton_mentioned_text = clinton_mentioned_negative['text_topicmodel'].apply(lambda x: nltk.pos_tag(x))
clinton_mentioned_text

0    [(cant, JJ), (one, CD), (til, NN), (read, VBD)...
0    [(clearly, RB), (top, JJ), (secret, JJ), (info...
0    [(randpaul, NN), (lying, VBG), (congress, NN),...
0    [(randpaul, NN), (lying, VBG), (congress, NN),...
0    [(randpaul, NN), (lying, VBG), (congress, NN),...
                           ...                        
0    [(always, RB), (angry, JJ), (dont, NN), (like,...
0    [(hillary, JJ), (get, NN), (focused, VBD), (me...
0    [(please, NN), (stand, VB), (standingrocksioux...
0    [(thank, NN), (clarifying, VBG), (hasnt, NN), ...
0    [(half, NN), (supporter, NN), (basket, NN), (d...
Name: text_topicmodel, Length: 2158, dtype: object

In [82]:
#Filter out the nouns for each row
clinton_mentioned_nouns = pd.DataFrame(np.zeros((len(clinton_mentioned_text), 1)))
clinton_mentioned_nouns.columns = ['Text']
for i in range(0, len(clinton_mentioned_text)):
    nouns = [word for (word, pos) in clinton_mentioned_text.iloc[i] if is_noun(pos)]
    nouns = ' '.join(nouns)
    clinton_mentioned_nouns.iloc[i] = nouns

In [83]:
#Split the Text 
clinton_mentioned_nouns['Text'] = clinton_mentioned_nouns['Text'].apply(lambda x: x.split())

In [84]:
clinton_text = clinton_mentioned_nouns.Text.tolist()
dictionary_clinton = corpora.Dictionary(clinton_text)
doc_term_matrix_clinton = [dictionary_clinton.doc2bow(doc) for doc in clinton_text]
ldamodel_clinton_2 = Lda_clinton(doc_term_matrix_clinton, num_topics=10, id2word = dictionary_clinton, passes=2)
ldamodel_clinton_2.print_topics(num_topics=10, num_words=5)

[(0,
  '0.041*"lie" + 0.025*"didnt" + 0.023*"ground" + 0.022*"troop" + 0.019*"medium"'),
 (1,
  '0.028*"r" + 0.023*"money" + 0.014*"something" + 0.014*"dont" + 0.013*"hillary"'),
 (2,
  '0.024*"vote" + 0.022*"dont" + 0.019*"plan" + 0.018*"shes" + 0.017*"cold"'),
 (3,
  '0.023*"hill" + 0.022*"imwithher" + 0.021*"fact" + 0.019*"people" + 0.016*"evil"'),
 (4,
  '0.036*"year" + 0.030*"education" + 0.025*"press" + 0.023*"conference" + 0.013*"death"'),
 (5,
  '0.032*"liar" + 0.030*"get" + 0.023*"people" + 0.019*"control" + 0.016*"show"'),
 (6,
  '0.032*"time" + 0.024*"men" + 0.019*"state" + 0.018*"america" + 0.016*"cause"'),
 (7,
  '0.026*"campaign" + 0.026*"office" + 0.022*"obama" + 0.020*"amp" + 0.020*"state"'),
 (8,
  '0.035*"dont" + 0.031*"people" + 0.027*"man" + 0.019*"anything" + 0.018*"speech"'),
 (9,
  '0.034*"reason" + 0.028*"friend" + 0.024*"job" + 0.023*"corruption" + 0.022*"youre"')]

In [85]:
topics_table_trump_po = pd.DataFrame(np.zeros((5, 10)), 
                            index = ['Trump - Pos', 'Trump - Pos', 'Trump - Pos', 'Trump - Pos', 'Trump - Pos'])
topics_table_trump_po.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10']

In [86]:
for i in range(0,10):
    for z in range(0,5):
        topics_table_trump_po.iloc[z,i] = ldamodel_trump_1.print_topics(num_topics=10, num_words=5)[i][1].split("+")[z]

In [87]:
topics_table_trump_neg = pd.DataFrame(np.zeros((5, 10)), 
                            index = ['Trump - Neg', 'Trump - Neg', 'Trump - Neg', 'Trump - Neg', 'Trump - Neg'])
topics_table_trump_neg.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10']

In [88]:
for i in range(0,10):
    for z in range(0,5):
        topics_table_trump_neg.iloc[z,i] = ldamodel_trump_2.print_topics(num_topics=10, num_words=5)[i][1].split("+")[z]

In [89]:
topics_table_clinton_po = pd.DataFrame(np.zeros((5, 10)), 
                            index = ['Clinton - Pos', 'Clinton - Pos', 'Clinton - Pos', 'Clinton - Pos', 'Clinton - Pos'])
topics_table_clinton_po.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10']

In [90]:
for i in range(0,10):
    for z in range(0,5):
        topics_table_clinton_po.iloc[z,i] = ldamodel_clinton_1.print_topics(num_topics=10, num_words=5)[i][1].split("+")[z]

In [91]:
topics_table_clinton_neg = pd.DataFrame(np.zeros((5, 10)), 
                            index = ['Clinton - Neg', 'Clinton - Neg', 'Clinton - Neg', 'Clinton - Neg', 'Clinton - Neg'])
topics_table_clinton_neg.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10']

In [92]:
for i in range(0,10):
    for z in range(0,5):
        topics_table_clinton_neg.iloc[z,i] = ldamodel_clinton_2.print_topics(num_topics=10, num_words=5)[i][1].split("+")[z]

In [112]:
output_topicmodel = pd.concat([topics_table_trump_po, topics_table_trump_neg, topics_table_clinton_po, topics_table_clinton_neg])

In [119]:
output_topicmodel.to_csv('output_topicmodel.csv', sep = ",") 

**Testing one tweet**

In [120]:
#Test specific document how it would have been classified 
bow_doc_1 = doc_term_matrix_trump[10]

In [121]:
for i in range(len(bow_doc_1)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_1[i][0], 
                                               dictionary_trump[bow_doc_1[i][0]], 
bow_doc_1[i][1]))

Word 39 ("day") appears 1 time.
Word 40 ("release") appears 1 time.
Word 41 ("sorry") appears 1 time.
Word 42 ("weapon") appears 1 time.


In [131]:
trump_mentioned_negative.iloc[10][['text', 'user_screen_name', 'created_at']]

text                @sethmoulton @realDonaldTrump will release nuc...
user_screen_name                                             karmakar
created_at                             Fri Aug 12 11:24:23 +0000 2016
Name: 0, dtype: object

In [132]:
trump_mentioned_negative.iloc[10]['text']

"@sethmoulton @realDonaldTrump will release nuclear weapons and the next day say 'sorry I was being sarcastic'"

In [123]:
for index, score in sorted(ldamodel_trump_2[bow_doc_1], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, ldamodel_trump.print_topic(index, 10)))

IndexError: index 4 is out of bounds for axis 0 with size 2

In [125]:
topics_table = pd.DataFrame(np.zeros((11, 10)))
topics_table.columns = ['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10']
topics_table

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
for i in range(0, 10):
    for z in range(0,10):
        topics_table.iloc[z,i] = (ldamodel_trump_2.print_topic(i, 10)).split("+")[z]

In [128]:
for index, score in sorted(ldamodel_trump_2[bow_doc_1], key=lambda tup: -1*tup[1]):
    for i in range(0,10): 
        topics_table.iloc[10,i] = ldamodel_trump_2[bow_doc_1][i][1]

In [130]:
topics_table.to_csv('topics_table.csv', sep = ",") 